**YOUR NAME HERE**

Spring 2020

CS 251B: Data Analysis and Visualization

Project 3: Linear regression

QR-based linear regression and polynomial regression

In [ ]:
import os
import random
import numpy as np
import matplotlib.pyplot as plt

import data
import linear_regression

plt.style.use(['seaborn-colorblind', 'seaborn-darkgrid'])
plt.rcParams.update({'font.size': 20})

np.set_printoptions(suppress=True, precision=5)

# Automatically reload external modules
%load_ext autoreload
%autoreload 2

## Notes

- In your implementations, only the following "high level" `scipy`/`numpy` functions can be used:
    - `np.linalg.inv`
    - `scipy.linalg.lstsq` (in `LinearRegression::linear_regression_scipy` only).
    - `np.linalg.norm`
    - (LA section only): `scipy.linalg.solve_triangular`, 

## Task 4: Implement a QR based linear regression solver

### 4a) QR-decomposition

- Write algorithm to compute QR decomposition (`linear_regression::qr_decomposition`). **Run test code below.** Equation for R: $$R = Q^TA$$
- Implement `linear_regression::linear_regression_qr` to use the QR decomposition to do the linear regression. Recall that the equation is $$Rc = Q^Ty$$ which can be solved without taking the inverse of $R$ through backsolving.

#### Test QR decomposition

In [ ]:
iris_filename = 'data/iris.csv'
iris_data = data.Data(iris_filename)

A = iris_data.select_data(['sepa_length', 'petal_width'])
A1 = np.hstack([A, np.ones([len(A), 1])])

lin_reg_qr = linear_regression.LinearRegression(iris_data)
myQ, myR = lin_reg_qr.qr_decomposition(A1)

Q, R = np.linalg.qr(A1)

print('NOTE: It is ok if everything is GLOBALLY negated.\n')
print(f'Your Q shape is {myQ.shape} and should be {Q.shape}')
print(f'Your R shape is {myR.shape} and should be {R.shape}')
print(f'1st few rows of your Q are\n{myQ[:3]} and should be\n{Q[:3]}')
print(f'\nYour R is\n{myR[:5]} and should be\n{R[:5]}')

#### Test linear regression via QR decomposition

In [ ]:
lin_reg_qr.linear_regression(['sepal_length'], 'petal_width', 'qr')
lin_reg_qr.scatter('sepal_length', 'petal_width', 'qr')
lin_reg_qr.show()

**Question 4:** How do your results compare to the built-in SciPy solver? Is this what you expected? Why or why not?

**Answer 4:**

### Task 4b) Compare performance of the linear regression methods

- Implement `mean_sse` in `linear_regression.py`. **Run test code below**
- In the cell below, load in the brain network `brain.csv` dataset.
- Create `Data` and `LinearRegression` objects.
- Load in the list of variable names in the brain dataset (62)
- Do multiple linear regressions, separately for each of the linear regression methods that you have implemented (including `scipy`): Set all brain data variables except for the last one as the independent variables, the last variable is the depenendent variable.
- Compute the mean sum of squares error (MSSE) in the predictions made by each linear regression model and the actual y values.

**Equation for MSSE:** $$E = \frac{1}{N}\sum_{i=1}^N \left (y_i - \hat{y}_i \right )^2$$

#### Test Mean SSE

In [ ]:
# Test with iris data
lin_reg_norm = linear_regression.LinearRegression(iris_data)
lin_reg_norm.linear_regression(['sepal_length'], 'petal_width', 'normal')
print(f'Your Iris mean SSE is {lin_reg_norm.mean_sse():.2f} and should be 0.19')

# Test with passing in random data
np.random.seed(0)
test_data = np.random.rand(iris_data.get_num_samples(), 1)
print(f'Your random data mean SSE is {lin_reg_norm.mean_sse(X=test_data):.2f} and should be 16.72')

**Question 5:** What are the mean sum of squares errors for each of the regression methods on the brain data? Which method(s) do best and which do the worst **and why**?

**Answer 5:**

## Task 5: Polynomial Regression

We have been creating linear regression fits of form $y =c_0 + c_1x_1 + c_2x_2 + \ldots$, where $x_i$ are independent variables (columns of $A$) and $c_i$ are corresponding coefficients in $c$. However, this equation only allows us to fit with a line/plane. This may not be the best choice for all datasets.

In this task, you will generalize the linear regression model form to include higher-degree (>1) polynomial terms and explore how this may improve fits to complex data. For example, assume we're doing a single linear regression with independent variable $x_1$ and dependent variable $y$. A linear regression that fits data with a quadratic shape has the form$$y = c_0 + c_1x_1 + c_2x_1^2$$ 

### 5a) Implement polynomial regression

- Implement two methods: `LinearRegression::poly_regression` and `LinearRegression::make_polynomial_matrix` (*helper method*). To implement polynomial regression, recall that you add one or more "independent variables" to $A$, derived from the original column vector $x_1$. For example, if $A$ originally has a column vector for the independent variable $\vec{x_1}$ ($A = [\vec{x_1}]$) and we wanted to make the above quadratic model, we would append $x_1^2$ ($A = [\vec{x_1}, \vec{x_1^2}]$).
- Add support for plotting polynomials in `LinearRegression::scatter`. To do this, you are going to generalize the regression line to a regression polynomial (if `self.p > 1`).
    - Getting your polynomial "x" values: Duplicate your line sample points for $p$ columns then raise each column to the appropriate power.
    - Getting your polynomial "y" values: Use matrix multiplication with your polynomial regression model slopes and/or intercepts.
- Update `predict` to run `make_polynomial_matrix` on `X` to change it if `X` is not `None` and `self.p > 1`.

#### Test `make_polynomial_matrix`

In [ ]:
test_A = np.r_[1:10].reshape((9, 1))
test_p = 3

# Test cubic
lin_reg = linear_regression.LinearRegression(data.Data())
print(f'Your polynomial matrix:\n{lin_reg.make_polynomial_matrix(test_A, 3)}')

true_mat = '''
[[  1.   1.   1.]
 [  2.   4.   8.]
 [  3.   9.  27.]
 [  4.  16.  64.]
 [  5.  25. 125.]
 [  6.  36. 216.]
 [  7.  49. 343.]
 [  8.  64. 512.]
 [  9.  81. 729.]]
'''
print('It should look like:\n', true_mat)

### 5b) Fitting data with polynomial regression

- In the cell below, load in `poly_data_1.csv`, make a `Data` object, make a `LinearRegression` object based off it like usual.

#### (i) Run polynomial regression on `poly_data_1` and plot results with `p=1`

- Run `poly_regression` with the polynomial degree $p = 1$. Solve using the normal equations.
- Use `scatter` to plot the results below.
- Print out the mean sum of squares error.

The plot created by running the below cell should "look right" to you.

#### (ii) Higher degree polynomials

In the cell below,
- Run `poly_regression` on the same datset with `p = 5`.
- Plot the results.
- Print out the mean squared error.

**Question 6:** Describe the fit compared to `p=1` — is it better or worse? Why?

**Question 7:** Describe what happens as you experiment with the polynomial degree between 1 and 5.

**Answer 6:**

**Answer 7:**

#### (iii) Check overfitting

Now you will check to see how the $p=5$ polynomial regression model generalizes to new data. 

- Load in `poly_data_2.csv`, which is the 2nd half of the `poly_data_1` dataset, make a new `Data` object, etc.
- Run `scatter` to show a scatterplot and the polynomial regression of the `poly_data_2` data, but use the polynomial regression model fit to the `poly_data_1`. Do this by replacing the `Data` object in `LinearRegression` object with the one holding `poly_data_2`. Use `set_data()`; do NOT run `poly_regression` again. You want to use the same fitted slopes/intercept from `poly_data_1`.
- Print out the mean sum of squared error obtained when predicting the `poly_data_2` using the regression model fit on `poly_data_1`. Do this by passing in the data column `X` selected from `poly_data_2` as a column vector ndarray.

**Question 8:** Describe how the fitted coefficients obtained for `poly_data_1` do on `poly_data_2`. Do they do a good or bad job? Why?

**Answer 8:**

#### (iv) Even higher degree polynomials

In the cell below, experiment with fitting and plotting `poly_data_1` again, but this time with an even higher degree polynomial `p=19`.

Be sure to:
- Fit the higher degree polynomial to the `poly_data_1` data.
- Plot the results for both `poly_data_1` and `poly_data_2`.
- Print out the MSSE for both `poly_data_1` and `poly_data_2`
- Like you did with `p = 5`, experiment how the results transfer to `poly_data_2` (i.e. do NOT re-run `poly_regression` after fitting coefficients to `poly_data_1`.

**Question 9:** What's going on with the higher degree polynomial fit and $R^2$?

**Question 10:** Compare the results for `poly_data_1` and `poly_data_2`. Back up your observations with numbers (e.g. MSSE).

**Answer 9:**

**Answer 10:**

## Extensions

To receive credit for any extension, you must:
- Not modify / prevent any code from the core project from working (e.g. make a copy before changing). In other words, **the notebook test code should still work!**
- **You must describe what you did and what you found in detail**. This includes a summary of parameter values used in your simulations.
- Include (*labeled!*) plots and/or numbers to present your results.
- Write up your extensions below or in a separate notebook.

**Rule of thumb: one deep, thorough extension is worth more than several quick, shallow extensions!**

**Reminder:** Give credit to all sources, including anyone that you consulted.

### 1. Your own data

- Run linear regression on datasets that interest you. Identify your hypotheses about the association between variables and test them out. Make plots and report all relevant metrics fit.

### 2. Linear regression algorithm comparison

- Compare the linear regression methods that you implemented on a dataset of your choice.
- Research and implement matrix condition number. Find a dataset with a poor matrix condition number and then compare the regression methods. Which does best and why?

### 3. Confidence intervals or other kinds of error bars on linear regression plots

- Add the option to plot 95% confidence intervals on the linear regression predictions in your plot functions (e.g. `scatter`).

### 4. Overfitting

- Run polynomial regression on other datasets. What degree polynomial works well? When do you overfit?